In [1]:
import os, uuid, json, requests
from datetime import datetime, timedelta, timezone
from pymongo import MongoClient
from dotenv import load_dotenv
from bson import json_util as bsonju

load_dotenv()

PREVIOUS_DOC = {
  "id": "9f3a27e6-1b52-4e0b-bc6e-62c1f2a7c8b9",
  "previous_form_qr_code": "20251011",
  "current_form_qr_code": "20251012",
  "date": "2025-10-01",
  "location": "JEF Gas Station – Sikatuna Branch",
  "tin": "123-456-789",
  "branch": "JEF Biosciences Fuel Division",
  "created": { "$date": "2025-10-12T08:00:00Z" },
  "items": [
    {"barcode":"4809992000101","type":"fuel","name":"Premium Gasoline","price":69.50,"unit":"liter","previous_quantity":520,"addstock":300,"sold":250,"current_quantity":570},
    {"barcode":"4809992000102","type":"fuel","name":"Regular Gasoline","price":64.75,"unit":"liter","previous_quantity":680,"addstock":400,"sold":320,"current_quantity":760},
    {"barcode":"4809992000103","type":"fuel","name":"Premium Diesel","price":61.20,"unit":"liter","previous_quantity":450,"addstock":350,"sold":280,"current_quantity":520},
    {"barcode":"4809992000201","type":"lubricant","name":"Engine Oil SAE 40","price":250.00,"unit":"liter","previous_quantity":60,"addstock":40,"sold":25,"current_quantity":75},
    {"barcode":"4809992000202","type":"lubricant","name":"Automatic Transmission Fluid (ATF)","price":310.00,"unit":"liter","previous_quantity":50,"addstock":30,"sold":20,"current_quantity":60},
    {"barcode":"4809992000203","type":"lubricant","name":"Gear Oil EP 90","price":285.00,"unit":"liter","previous_quantity":45,"addstock":25,"sold":15,"current_quantity":55},
    {"barcode":"4809992000204","type":"lubricant","name":"Hydraulic Oil ISO 68","price":295.00,"unit":"liter","previous_quantity":35,"addstock":25,"sold":10,"current_quantity":50},
    {"barcode":"4809992000205","type":"lubricant","name":"2T Motorcycle Oil","price":180.00,"unit":"liter","previous_quantity":80,"addstock":60,"sold":45,"current_quantity":95}
  ],
  "cashier": "Liam Santos",
  "cashier_employee_number": "50321",
  "recorder": "Ella Reyes",
  "recorder_employee_number": "50325",
  "is_empty": False
}

QR_API_URL = os.getenv(
    "GENERATE_EMPLOYEE_NUMBER_URL",
    "https://faas-sgp1-18bc02ac.doserverless.co/api/v1/web/fn-86117eee-d088-4b7a-9213-ab714825d87a/employees/generate-employee-number"
)

def _next_business_date(s):
    d = datetime.strptime(s, "%Y-%m-%d")
    return (d + timedelta(days=1)).strftime("%Y-%m-%d")

def _now_utc():
    return datetime.now(timezone.utc)

def _fetch_new_qr():
    try:
        r = requests.get(QR_API_URL, timeout=10)
        r.raise_for_status()
        data = r.json()
        # API returns {"ok": true, "employee_number": "57903"}; use it as the new QR code
        val = (data.get("current_form_qr_code")
               or data.get("employee_number"))
        if not val:
            raise ValueError("QR API response missing code field")
        return str(val)
    except Exception as e:
        # Fallback to a UUID suffix if API fails
        return uuid.uuid4().hex[:8]

def _roll_items(prev_items):
    now = _now_utc()
    rolled = []
    for idx, it in enumerate(prev_items, start=1):
        rolled.append({
            "item_number": str(idx),
            "id": str(uuid.uuid4()),
            "created": now,
            "updated": now,
            "barcode": it["barcode"],
            "type": it["type"],
            "name": it["name"],
            "price": it["price"],
            "unit": it["unit"],
            "previous_quantity": it.get("current_quantity", 0),
            "addstock": 0,
            "sold": 0,
            "current_quantity": 0
        })
    return rolled

def build_new_inventory(prev_doc):
    now = _now_utc()
    return {
        "id": str(uuid.uuid4()),
        "previous_form_qr_code": prev_doc["current_form_qr_code"],
        "current_form_qr_code": _fetch_new_qr(),
        "date": _next_business_date(prev_doc["date"]),
        "location": prev_doc["location"],
        "tin": prev_doc["tin"],
        "branch": prev_doc["branch"],
        "created": now,
        "items": _roll_items(prev_doc.get("items", [])),
        "is_empty": True
        # intentionally omit cashier/recorder fields for the new empty form
    }

new_doc = build_new_inventory(PREVIOUS_DOC)

uri = os.getenv("MONGODB_URI")
database = os.getenv("MONGODB_DATABASE")
collection_name = "inventories"

if uri and database:
    client = MongoClient(uri)
    coll = client[database][collection_name]
    result = coll.insert_one(new_doc)
    print(f"inserted_id: {result.inserted_id}")

print(bsonju.dumps(new_doc, ensure_ascii=False, indent=2))


inserted_id: 68eeca16b0026793153ff311
{
  "id": "41e35a41-a6ed-4c03-b1e5-306b8cdb554a",
  "previous_form_qr_code": "20251012",
  "current_form_qr_code": "39542",
  "date": "2025-10-02",
  "location": "JEF Gas Station – Sikatuna Branch",
  "tin": "123-456-789",
  "branch": "JEF Biosciences Fuel Division",
  "created": {
    "$date": 1760479764091
  },
  "items": [
    {
      "item_number": "1",
      "id": "d06b5ff5-5e47-457f-aaec-4df406b23150",
      "created": {
        "$date": 1760479766516
      },
      "updated": {
        "$date": 1760479766516
      },
      "barcode": "4809992000101",
      "type": "fuel",
      "name": "Premium Gasoline",
      "price": 69.5,
      "unit": "liter",
      "previous_quantity": 570,
      "addstock": 0,
      "sold": 0,
      "current_quantity": 0
    },
    {
      "item_number": "2",
      "id": "8d09aa53-9af3-4bd1-bae9-ff3b64e05187",
      "created": {
        "$date": 1760479766516
      },
      "updated": {
        "$date": 1760479766516
